In [1]:
import numpy as np
import pandas as pd

In [34]:
"""
A class that performs as one kind of stocks
"""

class Securities:
#     class level attributes
    
#     Initialized with daily closed price
#     price and shares are the present price and shares to initialized
#     preprice in the price of previous period 
    def __init__(self, name: str, price, shares):
        self.price = price
        self.shares = shares
        self.name = name
        setattr(self, 'preprice', price) # automatically initiate preprice and preshares attributes to calculate gains from this stock
        setattr(self, 'preshares', shares)
        setattr(self, 'modify', 0) # .modify is the attibute that indicate shares to modify
        
    @property
    def price(self):
        return self._price
        
    @price.setter
    def price(self,price):
        if price < 0:
            raise ValueError("Price of securities cannot be negative")
        self._price = price
        
# -----------------------------------------------------------------------
# setting methods
    def reset(self, new_price, shares_to_modify):
        self.preshares = self.shares
        if shares_to_modify < 0:
            if -shares_to_modify > self.shares:
                raise ValueError('Can not sell more shares than you have')
            else:
                self.shares += shares_to_modify
        else:
            self.shares += shares_to_modify
        self.preprice = self.price
        self.price = new_price
        self.modify = shares_to_modify
        
#     .gains() returns the total gains(or losses) from selling(or buying) certain shares of stock at the present price
    def gains(self):
        if self.modify == 0:
            raise ValueError('None shares need to modify.')
        shares_to_modify = -self.modify
        self.modify = 0 # important! reset the .modify attribute to be 0
        return self.price * shares_to_modify

In [242]:
"""
A class that performs as a securities account
"""
class Account:
#     class level attributes
    INITCASH = 1000000
    Tax = 0.25
    
#     stocks are one or several Securities objects
#     marval -- market value of the portfolio
    def __init__(self,*stocks):
        values = [*stocks] # portfoio of this account
        keys = [y.name for y in [*stocks]] # list of stocks' name in portfolio
        self.port = dict(zip(keys, values))
        marval = sum([y.price*y.shares for y in [*stocks]])
        if marval > self.INITCASH:
            raise ValueError('Cannot initiate account with total market value more than initial cash')
        else:
            setattr(self, 'cashleft', self.INITCASH - marval)
            setattr(self, 'cashpool', self.INITCASH - marval)
    
# -------------------------------------------------------------------------
# Start setting methods

#     update the market value in time
    @property
    def marval(self):
        marval = sum([x.price*x.shares for x in self.port.values()])
        return marval
    
    def __repr__(self):
        print("stock name | present price | present position")
        for stock_name in self.port.keys():
            print(f"{stock_name} \t\t {self.port[stock_name].price} \t\t {self.port[stock_name].shares}")
        return f'present market value: {self.marval}'
        
# The reallocation method will take a dict as input 
# The dict looks like {'AAPL':(10,0),'GOOGL':(12,100),'ORCL':(7.5,-100),...}, 
# keys of the dict are names of stocks to modify, for the dict-value tuple the first element is the new price and the second one is shares to modify
# The method has 3 functions:
#                             reset every Securities object that is to be modified
#                             calculate the gains and losses of the account
#                             calculate the yield of portfolio
    # adjusts is a list of daily shares and price adjustment
    def sell(self, adjusts: dict): 
        adjprice = dict(zip([x for x in adjusts.keys()], 
                            [y[0] for y in adjusts.values()]))
        adjshares = dict(zip([x for x in adjusts.keys()], 
                             [y[1] for y in adjusts.values()]))
        for adjust in adjshares.values():
            if adjust >= 0:
                raise ValueError('Shares to modify in the to-sell list must be negative!')
        for stock in adjusts.keys():
            if stock not in self.port.keys():
                raise ValueError('Illegal adjustment! Can not sell stocks not in the portfolio')

#          sell out all the stock in the to-sell list and cauculate gains
        for stock_name in adjusts.keys():
            new_price = adjprice[stock_name]
            shares_to_modify = adjshares[stock_name]
            stock = self.port[stock_name] # stock is a Securities object
            stock.reset(new_price, shares_to_modify)  # important command to reset stock
            gains = stock.gains()
#             This is the objective execution of this function 
            self.cashpool += gains # gains cash from selling stocks in the to-sell list
        soldout = list(stock for stock in self.port.values())
        for stock in soldout: # remove the stocks whose position have been closed from the portfolio
                if stock.shares == 0:
                    del self.port[stock.name]
        setattr(self, 'havesell', True)
    
    def buy(self, adjusts: dict):
        if not self.havesell:
            raise ValueError('You must sell some securities first obtaining some cash to support trading!')
        adjprice = dict(zip([x for x in adjusts.keys()], 
                            [y[0] for y in adjusts.values()]))
        adjshares = dict(zip([x for x in adjusts.keys()], 
                             [y[1] for y in adjusts.values()]))
        for stock_name in adjusts.keys():
            if adjshares[stock_name] <= 0:
                raise ValueError('Shares in the adjusts must be positive!')
            if stock_name not in self.port.keys():
                new_stock = Securities(stock_name, 
                                       adjprice[stock_name],
                                       adjshares[stock_name])
                self.port[stock_name] = new_stock
            stock = self.port[stock_name]
            new_price = adjprice[stock_name]
            modify = adjshares[stock_name]
            stock.reset(new_price, modify)
            gains = stock.gains()
            if self.cashpool + gains < 0:
                raise ValueError('Cannot use more money than cash pool!')
            self.cashpool += gains
        self.havesell = False
            
            

In [243]:
apple = Securities('AAPL',110,2000)
google = Securities('GOOGL',23,1000)
oracle = Securities('ORCL',52,3000)
amazon = Securities('AMZN',206,1200)
ibm = Securities('IBM',87,4000)
account = Account(apple,google,oracle,amazon,ibm)
repr(account)

stock name | present price | present position
AAPL 		 110 		 2000
GOOGL 		 23 		 1000
ORCL 		 52 		 3000
AMZN 		 206 		 1200
IBM 		 87 		 4000


'present market value: 994200'

In [212]:
account.cashpool
# account.havesell

5800

In [213]:
account.port

{'AAPL': <__main__.Securities at 0x10cbe8ac8>,
 'GOOGL': <__main__.Securities at 0x10cbe8da0>,
 'ORCL': <__main__.Securities at 0x10cbcc240>,
 'AMZN': <__main__.Securities at 0x10cbcc160>,
 'IBM': <__main__.Securities at 0x10cbccc88>}

In [214]:
adjusts = {'AAPL':(111,-1000),
           'GOOGL':(15,1000),
           'ORCL':(66,-3000),
           'AMZN':(190,500)}

In [215]:
adjusts_sell = dict(x for x in adjusts.items() if x[1][1]<0)
adjusts_buy = dict(x for x in adjusts.items() if x[1][1]>0)

In [216]:
adjusts_buy['BABA'] = (89, 2400)
adjusts_buy

{'GOOGL': (15, 1000), 'AMZN': (190, 500), 'BABA': (89, 2400)}

In [217]:
account.sell(adjusts_sell)

In [221]:
account.cashpool

204800

In [223]:
repr(account)

'<__main__.Account object at 0x10cbcc358>'

In [220]:
account.buy(adjusts_buy)

ValueError: Cannot use more money than cash pool!

In [209]:
account.cashpool

100

In [109]:
apple.gains()

-300